# Ruteo

Este notebook permite realizar la optimización de todos los pedidos del set de datos de inputs. El resultado será una asignación optimizada para cada columna de `pedidos` de la hoja de `pedidos` en el excel inputs.

## Preparación de la Sesión

Cargamos a la sesión de Google Colab el repositorio con los datos y los módulos desarrollados. Este paso **no** es necesario si se corre el notebook de manera local.

In [2]:
!git clone https://github.com/JuanCruzC97/ope3-logistica.git &> /dev/null

In [3]:
cd ope3-logistica

/content/ope3-logistica


In [4]:
!pip install -e . &> /dev/null

## Módulos

Importamos librerías y módulos propios para llevar a cabo la optimización.

In [5]:
# Importamos las principales librerías de manejo de datos y visualización.
import pandas as pd
import numpy as np
import plotly.express as px

# Importamos el componente principal Ruteo.
from logistica.ruteo import Ruteo
from logistica.utils import load_inputs, preparar_df_pedidos

# Importamos el módulo creado con metaheuristicas.
import logistica.metaheuristicas as mh

## Carga de Datos

Usamos los datos de inputs en `data/inputs/data_inputs.xlsx` con el formato requerido para incorporar los datos de pedidos y camiones al modelo.

### Subir Datos Externos

En caso de trabajar en Google Colab y querer subir datos que no se encuentren en el repositorio se deben correr la siguientes 2 celdas. Si trabajamos localmente directamente podemos agregar los datos a la carpeta de inputs.

In [103]:
# Solo correr esta celda si importamos datos externos de pedidos y camiones.
from google.colab import files
uploaded = files.upload()

Saving data_inputs.xlsx to data_inputs.xlsx


In [104]:
!rm /content/ope3-logistica/data/inputs/data_inputs.xlsx
!mv data_inputs.xlsx /content/ope3-logistica/data/inputs/data_inputs.xlsx

## Optimización

Se realiza un proceso completo de optimización para cada columna que contenga la palabra `pedido` en la hoja de `pedidos` del excel de inputs.

In [106]:
# Guardamos los datos de las hojas del input en dataframes.
df_pedidos = load_inputs("pedidos")
df_camiones = load_inputs("camiones")
df_parametros_ruteo = load_inputs("parametros_ruteo")
df_parametros_mh = load_inputs("parametros_mh")

if df_parametros_mh.random_state[0] == "None":
  random_state = None
else:
  random_state = df_parametros_mh.random_state[0]
  
if df_parametros_mh.max_time[0] == "None":
  max_time = None
else:
  max_time = df_parametros_mh.max_time[0]

pedidos_cols = [col for col in df_pedidos.columns if "pedido" in col]

results = None

for pedidos in pedidos_cols:

  df_pedido = preparar_df_pedidos(df_pedidos, pedidos)

  ruteo = Ruteo(df_camiones, df_pedido, costo_oportunidad=df_parametros_ruteo.costo_oportunidad[0], presupuesto=df_parametros_ruteo.presupuesto[0])
  ruteo.get_solucion_inicial(mode=df_parametros_mh.sol_inicial_mode[0], random_state=random_state)

  print(f'Optimización {pedidos}')
  best_sol, history = mh.sa(ruteo, 
                            t_inicial=df_parametros_mh.sa_t_inicial[0],
                            t_final=df_parametros_mh.sa_t_final[0], 
                            k=df_parametros_mh.sa_k[0], 
                            iters=df_parametros_mh.sa_iters[0], 
                            temp_mode=df_parametros_mh.sa_temp_mode[0], 
                            max_time=max_time,
                            prob=df_parametros_mh.vecinos_p[0], 
                            random_state=random_state)
  print("\n")

  df_sol = best_sol.summary_ruteo(time=history.get("time"), iters=history.get("iters"))
  df_sol.columns = [pedidos]
  df_sol_pedidos = best_sol.summary_pedidos()[["Camion"]]
  df_sol_pedidos.columns = [pedidos]

  if results is None:
    results = df_sol
    result_pedidos = df_sol_pedidos
  
  else:
    results = pd.concat([results, df_sol], axis=1)
    result_pedidos = pd.concat([result_pedidos, df_sol_pedidos], axis=1)

Optimización pedidos1


100%|██████████| 150/150 [00:03<00:00, 49.32it/s]




Optimización pedidos2


100%|██████████| 150/150 [00:03<00:00, 47.73it/s]




Optimización pedidos3


...

100%|██████████| 150/150 [00:03<00:00, 47.30it/s]




Optimización pedidos5


100%|██████████| 150/150 [00:03<00:00, 37.51it/s]




Optimización pedidos6


100%|██████████| 150/150 [00:03<00:00, 39.09it/s]

## Resultados

In [108]:
# Resultados generales.
results

,pedidos1,pedidos2,pedidos3,pedidos4,pedidos5,pedidos6
Carga Total,47.00,54.00,55.00,56.00,66.00,61.00
Costo Camiones,51600.00,56800.00,59000.00,57600.00,66000.00,61000.00
Costo Oportunidad,0.00,0.00,0.00,0.00,0.00,0.00
Costo Total,51600.00,56800.00,59000.00,57600.00,66000.00,61000.00
Costo Total por tn,1097.87,1051.85,1072.73,1028.57,1000.00,1000.00
Ahorro,-10.01,-13.78,-12.07,-15.69,-18.03,-18.03
Tiempo,3.05,3.15,3.10,3.19,4.01,3.85
Iteraciones,3750.00,3750.00,3750.00,3750.00,3750.00,3750.00


In [99]:
# Asignación de pedidos.
result_pedidos

,pedidos1,pedidos2,pedidos3,pedidos4,pedidos5,pedidos6
Pedido A,8.0,4.0,8.0,9.0,2,5.0
Pedido B,8.0,5.0,NaN,4.0,8,3.0
Pedido C,5.0,2.0,4.0,2.0,8,NaN
Pedido D,9.0,1.0,6.0,7.0,4,8.0
Pedido E,4.0,8.0,8.0,9.0,4,6.0
Pedido F,3.0,4.0,2.0,9.0,2,6.0
Pedido G,2.0,7.0,2.0,NaN,9,9.0
Pedido H,4.0,7.0,7.0,3.0,9,3.0
Pedido I,7.0,1.0,2.0,5.0,5,8.0
Pedido J,6.0,9.0,9.0,1.0,6,7.0


In [111]:
# Guardar los resultados.
with pd.ExcelWriter("data/outputs/data_outputs.xlsx") as writer:
    results.to_excel(writer, sheet_name="ruteo")
    result_pedidos.to_excel(writer, sheet_name="pedidos")